```{contents}
```
## ConversationBufferWindowMemory 


**ConversationBufferWindowMemory** stores **only the last *N* turns** of a conversation (a sliding window) and discards older messages.
It preserves **recent context** while keeping **token usage bounded**.

Provided by LangChain.

```
User ↔ Assistant
   ↓
Last N Messages (Window)
   ↓
Prompt (recent context only)
```

---

### Why ConversationBufferWindowMemory Is Needed

* Full buffer memory grows unbounded
* Summary memory may lose details
* Many tasks only need **recent context**

This memory is a **middle ground**:

* Keeps verbatim messages
* Limits history size
* Avoids summarization overhead

---

### How It Works Internally

1. Each user/assistant message is appended
2. If total messages exceed the window size:

   * Oldest messages are removed
3. Only the most recent **k turns** are injected into the prompt

Example with `k=2`:

```
[Kept]
Human: It disconnects every 5 minutes
AI: Try checking network drivers
Human: I already updated them
AI: Thanks for confirming

[Discarded]
Earlier conversation…
```

---

### Architecture View

![Image](https://media.geeksforgeeks.org/wp-content/uploads/20250926150233170952/architecture_of_conversation_buffer_window_memory.webp?utm_source=chatgpt.com)

![Image](https://media.geeksforgeeks.org/wp-content/uploads/20250925181819723356/features_of_conversation_buffer_memory.webp?utm_source=chatgpt.com)

![Image](https://timwappat.info/content/images/2024/12/Context-Window-Summary-1.jpg?utm_source=chatgpt.com)

---

### Basic Demonstration (LangChain)

#### Initialize Window Memory



In [1]:
from langchain_classic.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

C:\Users\sangouda\AppData\Local\Temp\ipykernel_22704\1644586139.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2)




Here, `k=2` means:

* Keep the **last 2 conversational turns**
* A turn = Human + AI messages

---

#### Attach Memory to a Conversation Chain



In [2]:
from langchain_classic.chains import ConversationChain
from langchain_openai import OpenAI

llm = OpenAI()

conversation = ConversationChain(
    llm=llm,
    memory=memory
)


C:\Users\sangouda\AppData\Local\Temp\ipykernel_22704\2335344415.py:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(




---

#### Run a Multi-Turn Conversation


In [3]:
conversation.predict(input="My VPN is not working")
conversation.predict(input="It disconnects every 5 minutes")
conversation.predict(input="I already tried restarting")
conversation.predict(input="What did I try earlier?")


' You mentioned restarting your device and reconnecting to the VPN. Sometimes, simply disconnecting and reconnecting to the VPN can also help resolve connectivity issues. Additionally, checking for software updates and reaching out to your VPN provider for assistance are also steps you have taken.'



**Expected Output**

```
You tried restarting the VPN.
```

The model recalls this because it is **within the window**.

If the window were exceeded, earlier details would be forgotten.

---

### Inspecting the Memory Buffer

```python
print(memory.buffer)
```

Example buffer (k=2):

```
Human: It disconnects every 5 minutes
AI: Have you tried restarting?

Human: I already tried restarting
AI: Thanks for confirming.
```

Older messages are **no longer present**.

---

### Prompt Injection Mechanism

The prompt includes only recent messages:

```
System: You are a helpful assistant.

Recent Conversation:
Human: I already tried restarting
AI: Thanks for confirming.

Human: What did I try earlier?
AI:
```

---

### Strengths of ConversationBufferWindowMemory

* Fixed token usage
* No summarization cost
* Preserves exact wording (recent)
* Simple and predictable behavior

---

### Limitations

| Limitation            | Explanation                |
| --------------------- | -------------------------- |
| Forgets older context | Hard cutoff                |
| No long-term memory   | Not suitable alone         |
| Manual tuning         | Window size must be chosen |
| No semantic recall    | Exact text only            |

---

### When to Use ConversationBufferWindowMemory

Use it when:

* Only recent context matters
* Conversations are task-focused
* You want low cost and simplicity
* You need deterministic memory behavior

Avoid it when:

* Long-term context is required
* Earlier facts must persist
* Compliance or audit trails are needed

---

### Comparison with Other Memory Types

| Memory Type                    | Stores       | Token Growth | Best Use        |
| ------------------------------ | ------------ | ------------ | --------------- |
| ConversationBufferMemory       | Full chat    | High         | Short chats     |
| ConversationBufferWindowMemory | Last N turns | Fixed        | Task-based chat |
| ConversationSummaryMemory      | Summary      | Low          | Long sessions   |
| VectorStoreMemory              | Embeddings   | Low          | Semantic recall |

---

### Real-World Use Case

**IT Support Assistant**

* Recent troubleshooting steps matter most
* Old greetings are irrelevant
* Window memory keeps the agent focused
* Often combined with vector or DB memory

---

### Key Takeaways

* ConversationBufferWindowMemory is a **sliding-window memory**
* Keeps recent turns verbatim
* Controls cost and context size
* Ideal for task-oriented conversational agents
